<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

## **What is systemd?**

`systemd` is a system and service manager for Linux that initializes the system and manages services. It uses unit files to define services, targets, and other system components.

## **Key Concepts**

1. **Unit Files**:
   - The configuration files used by `systemd`.
   - Common types of unit files:
     - **`.service`**: Describes a service.
     - **`.target`**: Groups units into logical targets (e.g., `multi-user.target`).
     - **`.timer`**: Schedules tasks.
     - **`.mount`**: Manages filesystem mounts.

2. **Unit File Locations**:
   - **System-wide**: `/etc/systemd/system/`
   - **Default**: `/usr/lib/systemd/system/`
   - **User-specific**: `~/.config/systemd/user/`

3. **Service States**:
   - `active`: Running.
   - `inactive`: Stopped.
   - `failed`: Encountered an error.

## **Creating a Service**

### **1. Writing a Service File**

Example: `my-service.service`

```ini
[Unit]
Description=My Custom Service
After=network.target

[Service]
Type=simple
ExecStart=/path/to/your/application --arg1 --arg2
WorkingDirectory=/path/to/working/directory
User=myuser
Restart=always
RestartSec=5

[Install]
WantedBy=multi-user.target
```

### **2. Key Sections in a Service File**
- **`[Unit]`**:
  - Describes the service and its dependencies.
  - `Description`: Short description of the service.
  - `After`: Specifies when the service should start (e.g., after `network.target`).

- **`[Service]`**:
  - Describes how the service operates.
  - `Type`: Defines the service type:
    - `simple`: (default) Starts the process specified in `ExecStart`.
    - `forking`: Used for daemons that fork.
    - `oneshot`: For one-time tasks.
    - `notify` or `dbus`: For advanced use cases.
  - `ExecStart`: Command to start the service.
  - `WorkingDirectory`: Sets the working directory.
  - `User`: User account under which the service runs.
  - `Restart`: Specifies restart behavior (`always`, `on-failure`, etc.).
  - `RestartSec`: Delay between restart attempts.

- **`[Install]`**:
  - Defines how and when the service is enabled.
  - `WantedBy`: Defines the target in which the service is included (e.g., `multi-user.target` for most services).



### **3. Steps to Enable and Start a Service**

1. **Place the Service File**:
   ```bash
   sudo cp my-service.service /etc/systemd/system/
   ```

2. **Reload systemd**:
   ```bash
   sudo systemctl daemon-reload
   ```

3. **Enable the Service** (Start at Boot):
   ```bash
   sudo systemctl enable my-service.service
   ```

4. **Start the Service**:
   ```bash
   sudo systemctl start my-service.service
   ```

5. **Check Status**:
   ```bash
   sudo systemctl status my-service.service
   ```

## **Managing Services**

1. **Basic Commands**:
   - Start a service:
     ```bash
     sudo systemctl start my-service
     ```
   - Stop a service:
     ```bash
     sudo systemctl stop my-service
     ```
   - Restart a service:
     ```bash
     sudo systemctl restart my-service
     ```
   - Reload service configuration without restarting:
     ```bash
     sudo systemctl reload my-service
     ```
   - Check service status:
     ```bash
     sudo systemctl status my-service
     ```

2. **Enable/Disable Services**:
   - Enable a service (start at boot):
     ```bash
     sudo systemctl enable my-service
     ```
   - Disable a service:
     ```bash
     sudo systemctl disable my-service
     ```

3. **Inspect Logs**:
   - View logs for a specific service:
     ```bash
     journalctl -u my-service
     ```
   - Tail logs in real-time:
     ```bash
     journalctl -u my-service -f
     ```

4. **Remove a Service**:
   - Stop the service:
     ```bash
     sudo systemctl stop my-service
     ```
   - Disable the service:
     ```bash
     sudo systemctl disable my-service
     ```
   - Delete the service file:
     ```bash
     sudo rm /etc/systemd/system/my-service.service
     ```
   - Reload systemd:
     ```bash
     sudo systemctl daemon-reload
     ```

## **Advanced Topics**

### **1. Environment Variables**
You can pass environment variables to a service using `Environment=` or `EnvironmentFile=`.

Example:
```ini
Environment=MY_VAR=value
EnvironmentFile=/path/to/env/file
```

---

### **2. Dependency Management**
- **`Requires`**: Specifies hard dependencies. If the required unit fails, this unit also fails.
- **`Wants`**: Specifies soft dependencies. If the dependency fails, this unit still runs.
- **`Before`/`After`**: Specifies the order of startup.

Example:
```ini
[Unit]
Requires=network.target
After=network.target
```

---

### **3. Timer Units**
Use `.timer` units to schedule tasks, similar to `cron`.

Example: `my-service.timer`
```ini
[Unit]
Description=Run My Service Every Day

[Timer]
OnCalendar=daily
Persistent=true

[Install]
WantedBy=timers.target
```

---

### **4. User Services**
Systemd also supports per-user services that don’t require root.

1. Place the service file in `~/.config/systemd/user/`.
2. Reload user services:
   ```bash
   systemctl --user daemon-reload
   ```
3. Enable and start the service:
   ```bash
   systemctl --user enable my-service
   systemctl --user start my-service
   ```

---

### **5. Debugging Services**
- Check for syntax errors in the service file:
  ```bash
  sudo systemd-analyze verify /etc/systemd/system/my-service.service
  ```
- Debug logs:
  ```bash
  sudo journalctl -xe
  ```
- Test the service manually:
  ```bash
  sudo -u myuser /path/to/application --arg1 --arg2
  ```

### **Best Practices**

1. **Use Absolute Paths**:
   Systemd requires absolute paths for all file and directory references.
   
2. **Limit Privileges**:
   Run services as non-root users wherever possible.

3. **Enable Restart Policies**:
   Use `Restart=always` or `Restart=on-failure` for critical services.

4. **Use `systemctl edit` for Overrides**:
   To modify system services without editing the main service file:
   ```bash
   sudo systemctl edit my-service
   ```
   This creates a drop-in configuration in `/etc/systemd/system/my-service.d/`.
